In [141]:
import matplotlib.pyplot as plt
import time

# Tensorflow / Keras
from keras import Model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import TimeDistributed
from keras.layers import Input
from keras.optimizers import Adam

from keras.optimizers import SGD

import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell

# Data manipulation
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Masking
from sklearn.metrics import accuracy_score
# file processing
import os

## Load Data

In [142]:
def slice_per_step(a, step=16):
    # add -1 to the end of each sample to make them the same length per step(piece_length)
    pad_count = step - (a.shape[0] % step)
    print(a.shape, end=' ')
    if pad_count!=step : 
        print('pad by',pad_count, end=' ')
        a = np.pad(a, ((0, pad_count), (0, 0)), 'constant', constant_values=-1)

    # reshape into per step
    a = np.reshape(a, (-1, step, a.shape[1]))
    print('to',a.shape)

    return a

In [143]:
# Load Data
PATH = "test_input"  
file_dir = os.listdir(PATH) 
X_data = list()
y_data = list()
max_len = -1

for file in file_dir:
    loaded_file = np.load(PATH + "/" + file)
    # if the file name ends with "X.npy"
    if file.endswith("ans.npy"):
        y_data.extend(slice_per_step(loaded_file))
    else:
        X_data.extend(slice_per_step(loaded_file))

X_data = np.array(X_data)
y_data = np.array(y_data)

print("X_data shape: ", X_data.shape, X_data.dtype)
print("y_data shape: ", y_data.shape, y_data.dtype)

(2420, 52) pad by 12 to (152, 16, 52)
(2420, 42) pad by 12 to (152, 16, 42)
(2256, 52) to (141, 16, 52)
(2256, 42) to (141, 16, 42)
(1800, 52) pad by 8 to (113, 16, 52)
(1800, 42) pad by 8 to (113, 16, 42)
X_data shape:  (406, 16, 42) float64
y_data shape:  (406, 16, 52) int64


## Parameter

In [144]:
n_step = len(X_data)
piece_length = 16

n_hidden = 200 # Hidden layer
n_feature = 42 # num of input features (12*3 + 1 + 1 + 2 + 1 + 1)
n_pitch = 52 # one-hot of pitch (B0 ~ D5)  

# Training
epoch = 100
learning_rate = 0.001
training_iters = n_step * epoch  # Loop epoch times on the dataset
batch_size = 64

print("Minimum data:", (n_step/batch_size)*epoch)

Minimum data: 634.375


## Split Data

In [145]:
# Data Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.1, random_state=42, shuffle=True)

print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)

print("y_train shape: ", y_data.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (365, 16, 42)
X_test shape:  (41, 16, 42)
y_train shape:  (406, 16, 52)
y_test shape:  (41, 16, 52)


## Additional Functions

In [146]:
def batch_generator(X, y):
    """ Input Output batch generator.
        
        return: 
        - 3d-array with shape of (batch_size, piece_length, n_feature)
        - 3d-array with shape of (batch_size, piece_length, n_feature)
    """
    while True:
        for i in range(0, n_step, batch_size):
            yield X[i:i+batch_size], y[i:i+batch_size]

## Build Neural Network

In [147]:
print("Creating model")
input = Input(shape=(piece_length, n_feature,), name="Input_step_of_3chromagram_speed_nInstrument_timeSig_beatPos_barPos")  

masked_input = Masking(mask_value=-1, name="Mask_layer_for_padded_data")(input) # Ignore Padded Data

LSTM_forward = LSTM(units=n_hidden,
                input_shape=(piece_length, n_feature, ),
                return_sequences=True,
                name="LSTM_forwards")(masked_input)

time_dist = TimeDistributed(Dense(n_pitch, activation="tanh"), name="Dense_layer_for_each_memory_cell")(LSTM_forward)

softmax = Activation('softmax', name="Softmax")(time_dist)

model = Model(inputs=input, outputs=softmax)

model.compile(loss='categorical_crossentropy',
                optimizer=Adam(learning_rate=learning_rate),
                metrics=['categorical_accuracy'])
print(model.summary())

Creating model
Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_step_of_3chromagram_s  [(None, 16, 42)]         0         
 peed_nInstrument_timeSig_be                                     
 atPos_barPos (InputLayer)                                       
                                                                 
 Mask_layer_for_padded_data   (None, 16, 42)           0         
 (Masking)                                                       
                                                                 
 LSTM_forwards (LSTM)        (None, 16, 200)           194400    
                                                                 
 Dense_layer_for_each_memory  (None, 16, 52)           10452     
 _cell (TimeDistributed)                                         
                                                                 
 Softmax (Activation)        (None, 16, 52)

## Train Model

In [148]:
# train model
model.fit_generator(batch_generator(X_train, y_train),
                    steps_per_epoch=(n_step/batch_size),
                    epochs=epoch,
                    verbose=1)
model.save("lstm")

Epoch 1/100


C:\Users\kevin\AppData\Local\Temp/ipykernel_5816/520341286.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(batch_generator(X_train, y_train),


6/6 [==============================] - 4s 44ms/step - loss: 3.6800 - categorical_accuracy: 0.1414
Epoch 2/100
6/6 [==============================] - 0s 45ms/step - loss: 3.1564 - categorical_accuracy: 0.3928
Epoch 3/100
6/6 [==============================] - 0s 45ms/step - loss: 2.9650 - categorical_accuracy: 0.4120
Epoch 4/100
6/6 [==============================] - 0s 43ms/step - loss: 2.8432 - categorical_accuracy: 0.4229
Epoch 5/100
6/6 [==============================] - 0s 44ms/step - loss: 2.7440 - categorical_accuracy: 0.4306
Epoch 6/100
6/6 [==============================] - 0s 44ms/step - loss: 2.6591 - categorical_accuracy: 0.4227
Epoch 7/100
6/6 [==============================] - 0s 43ms/step - loss: 2.5885 - categorical_accuracy: 0.4842
Epoch 8/100
6/6 [==============================] - 0s 43ms/step - loss: 2.5268 - categorical_accuracy: 0.5924
Epoch 9/100
6/6 [==============================] - 0s 42ms/step - loss: 2.4761 - categorical_accuracy: 0.6667
Epoch 10/100
6/6 [====

INFO:tensorflow:Assets written to: lstm\assets


INFO:tensorflow:Assets written to: lstm\assets


## Predict

In [149]:
pred_test = model.predict(X_test)

2/2 [==============================] - 1s 6ms/step
